In [2]:
# prompt: using shapely, create a .geojson file for each value in in beverly_hills_circuits.csv (aggregate_geometry, buffer_geometry, gesw_segment_geometry) use circuit_floc_id as the unique id. then merge them all into one KMZ file using arcpy.

%pip install shapely geopandas
import geopandas as gpd
from shapely.geometry import shape
import pandas as pd
import json

# Assuming beverly_hills_circuits.csv is in the current directory
df = pd.read_csv(r"C:\Users\mcveydb\Downloads\beverly_hills_circuits.csv")

# Function to create GeoJSON files
def create_geojson(row):
    # Extract geometries
    # if aggregate_geometry value type is string
    if type(row['aggregate_geometry']) == str:
        aggregate_geometry = shape(json.loads(row['aggregate_geometry']))
    else:
        aggregate_geometry = row['aggregate_geometry']
    if type(row['buffer_geometry']) == str:
        buffer_geometry = shape(json.loads(row['buffer_geometry']))
    else:
        buffer_geometry = row['buffer_geometry']
    if type(row['gesw_segment_geometry']) == str:
        gesw_segment_geometry = shape(json.loads(row['gesw_segment_geometry']))
    else:
        gesw_segment_geometry = row['gesw_segment_geometry']
    
    # Create GeoDataFrames
    gdf_aggregate = gpd.GeoDataFrame(geometry=[aggregate_geometry], index=[row['igpe_section_name']])
    gdf_buffer = gpd.GeoDataFrame(geometry=[buffer_geometry], index=[row['igpe_section_name']])
    gdf_gesw = gpd.GeoDataFrame(geometry=[gesw_segment_geometry], index=[row['igpe_section_name']])
    
    # Save to GeoJSON files
    gdf_aggregate.to_file(f"geojson/{row['igpe_section_name']}_aggregate.geojson", driver='GeoJSON')
    gdf_buffer.to_file(f"geojson/{row['igpe_section_name']}_buffer.geojson", driver='GeoJSON')
    gdf_gesw.to_file(f"geojson/{row['igpe_section_name']}_gesw.geojson", driver='GeoJSON')

# Apply the function to each row which contains a value in the 'circuit_floc_id' column
# if 'circuit_floc_id' value is not null, then apply the function
# df.loc[df['circuit_floc_id'].notnull(), 'circuit_floc_id'].apply(create_geojson)


df.apply(create_geojson, axis=1)


DEPRECATION: Loading egg at c:\users\mcveydb\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages\aiohttp-3.8.4-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mcveydb\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages\anthropic-0.3.6-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\mcveydb\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local-packages\python311\site-packages\beautifulsoup4-4.12.2-py3.11.egg is deprecated.

Note: you may need to restart the kernel to use updated packages.


C:\Users\mcveydb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
C:\Users\mcveydb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
C:\Users\mcveydb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(
C:\Users\mcveydb\AppData\Local\Packages\

0      None
1      None
2      None
3      None
4      None
       ... 
679    None
680    None
681    None
682    None
683    None
Length: 684, dtype: object

In [4]:

# For merging and creconverting to KMZ, you'll need arcgispro-py3 for arcpy

import arcpy
import os

# set the workspace to the geojson folder
arcpy.env.workspace = r"geojson"

geojson_files = arcpy.ListFiles("*_buffer.geojson")

# use tempfile to define a temporary folder to store intermediate shapefiles
import tempfile
temp_folder = tempfile.mkdtemp()
print(temp_folder)

# List to store converted shapefiles
shapefiles = []

# Loop through each GeoJSON file and convert to shapefile
for geojson_file in geojson_files:
    # Define the output shapefile path
    output_shapefile = os.path.join(temp_folder, os.path.splitext(geojson_file)[0] + ".shp")
    
    # Convert GeoJSON to shapefile
    arcpy.JSONToFeatures_conversion(geojson_file, output_shapefile)
    
    # Add the shapefile to the list
    shapefiles.append(output_shapefile)

# Merge shapefiles into one
merged_output = os.path.join(temp_folder, "merged_circuits.shp")
arcpy.management.Merge(shapefiles, merged_output)

# Export the merged shapefile to KMZ
kmz_output = r"merged_circuits.kmz"
arcpy.LayerToKML_conversion(merged_output, kmz_output)

print("GeoJSON files successfully merged and exported to KMZ.")


C:\Users\mcveydb\AppData\Local\Temp\tmpnqkkybrs
C:\Users\mcveydb\AppData\Local\Temp\tmp5nioau34
C:\Users\mcveydb\AppData\Local\Temp\tmpsvnxwel3


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000584: Implementation of this Tool's Validate is invalid.
Failed to execute (LayerToKML).


In [5]:

# For merging into a KMZ file, you'll need ArcGIS Pro and the arcpy library.

import arcpy
import os

# set the workspace to the geojson folder
arcpy.env.workspace = r"C:\Users\mcveydb\dev\GIS-Utilities\src\arcgis pro\geojson"

for type in ["aggregate", "buffer", "gesw"]:
    geojson_files = arcpy.ListFiles(f"*_{type}.geojson")

    # use tempfile to define a temporary folder to store intermediate shapefiles
    import tempfile
    temp_folder = tempfile.mkdtemp()
    print(temp_folder)

    # List to store converted shapefiles
    shapefiles = []

    # Loop through each GeoJSON file and convert to shapefile
    for geojson_file in geojson_files:
        # Define the output shapefile path
        output_shapefile = os.path.join(temp_folder, os.path.splitext(geojson_file)[0] + ".shp")
        
        # Convert GeoJSON to shapefile
        arcpy.JSONToFeatures_conversion(geojson_file, output_shapefile)
        
        # Add the shapefile to the list
        shapefiles.append(output_shapefile)

    # Merge shapefiles into one
    merged_output = os.path.join(temp_folder, f"merged_{type}.shp")
    arcpy.management.Merge(shapefiles, merged_output)

    # Export the merged shapefile to KMZ
    kmz_output = r"merged_{type}.kmz"
    arcpy.LayerToKML_conversion(merged_output, kmz_output)

    print(f"{type} GeoJSON files successfully merged and exported to KMZ.")


SyntaxError: unterminated string literal (detected at line 23) (2340543249.py, line 23)